In [29]:
!pip install pandas-datareader
!pip install fix_yahoo_finance
import pandas as pd
import datetime
from datetime import timedelta
import pandas_datareader.data as web
import fix_yahoo_finance as yf
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
class EventStudy_Fed():
    def __init__(self,file_path):
        try:
            pd.read_csv(file_path)
        except TypeError:
            return f'Wrong File Type.'
        self.dataframe = pd.DataFrame(pd.read_csv(file_path))
        self.company = self.dataframe['Ticker']
        self.market = self.dataframe['Market']
        self.industry = self.dataframe['Industry']
        self.year = self.dataframe['Year']
        self.month = self.dataframe['Month']
        self.day = self.dataframe['Day']
        self.dict = {}
        self.reg_result = list()
        self.fed_cumu_ab_ret_per_event = list()
        
        for i in range(0,len(self.market)):
            self.dict[self.company[i]] = self.market[i]
        eventdate = []
        for i in range(0,len(self.dataframe)):
            eventdate.append(datetime.datetime(self.year[i],self.month[i],self.day[i]))
        self.dataframe['EventDate'] = eventdate
        #suppose there is only one event in a month
        uniquedays = self.dataframe['EventDate'].unique()
        inter_days = []
        for i in range(0,len(uniquedays)-1):
            inter_days.append(int((uniquedays[i+1]-uniquedays[i])/np.timedelta64(1,'D')))
        self.sample_length = min(inter_days)
    
    def reg(self):
        eventdate_set = self.dataframe['EventDate'].unique()
        subdate_cumulative_abnormal_return = pd.DataFrame()
        for date in eventdate_set:
            subevent_df = self.dataframe[self.dataframe.EventDate == date]
            start_date = date - np.timedelta64(self.sample_length, 'D')
            end_date = date + np.timedelta64(self.sample_length, 'D')
            subevent_company_list = subevent_df['Ticker'].unique()
            subevent_cumulative_abnormal_return = pd.DataFrame()
            
            for company in subevent_company_list:
                market = subevent_df[subevent_df.Ticker == company]['Market'].iloc[0]
                price = web.DataReader([company, market],'yahoo', start_date,end_date)
                close_price = price['Close']
                return_rate = close_price.pct_change()
                x = return_rate[company]    
                X = sm.add_constant(x)
                y = return_rate[market]
                model = sm.OLS(y,X,missing='drop')
                result = model.fit()
#                 print(result.summary())
                (a,b) = result.params

                return_rate['Abnormal_Return'] = return_rate.apply(lambda x: x[market] - a - b * x[company], axis=1)
                subevent_cumulative_abnormal_return[f'{company}'] = return_rate['Abnormal_Return'].cumsum()[1:]
            subevent_cumulative_abnormal_return['Average_CAR'] = subevent_cumulative_abnormal_return.apply(lambda x: x.sum()/(len(subevent_company_list)),axis = 1)
                               

            self.reg_result.append(subevent_cumulative_abnormal_return['Average_CAR'])
            self.fed_cumu_ab_ret_per_event.append(subevent_cumulative_abnormal_return['Average_CAR'][-1])
    def visualization(self):
        COL_NUM = 2
        ROW_NUM = len(self.reg_result)//2+1
        fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(200,200))

        for i in range(len(self.reg_result)):
            ax = axes[int(i/COL_NUM), i%COL_NUM]
            self.reg_result[i].plot(kind='bar', ax=ax)

        plt.bar(list(range(8)),fed_cumu_ab_ret_per_event, align='center')
        plt.xticks(list(range(8)), fed_event_list)
        plt.xticks(rotation=90)
        plt.title("Portfolio Cumulative Abnormal Return for 8 Fed Events")
        plt.ylabel('return %');
        plt.show()
        
        fig1, ax1 = plt.subplots()
        ax1.pie(us_unique_industry_count_pct, labels=us_unique_industry_list, autopct='%1.1f%%',
        shadow=True, startangle=90)
        ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
        plt.show()

In [ ]:
!pip install pandas-datareader
!pip install fix_yahoo_finance
import pandas as pd
import datetime
from datetime import timedelta
import pandas_datareader.data as web
import fix_yahoo_finance as yf
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
class EventStudy_TradeWar():
    def __init__(self,file_path):
        try:
            pd.read_csv(file_path)
        except TypeError:
            return f'Wrong File Type.'
        self.dataframe = pd.DataFrame(pd.read_csv(file_path))
        self.company = self.dataframe['Ticker']
        self.market = self.dataframe['Market']
        self.industry = self.dataframe['Industry']
        self.year = self.dataframe['Year']
        self.month = self.dataframe['Month']
        self.day = self.dataframe['Day']
        self.dict = {}
        self.reg_result = list()
        self.reg_result_us = list()
        self.reg_result_china = list()
        self.us_return = 0
        self.china_return = 0
        
        for i in range(0,len(self.market)):
            self.dict[self.company[i]] = self.market[i]
        eventdate = []
        for i in range(0,len(self.dataframe)):
            eventdate.append(datetime.datetime(self.year[i],self.month[i],self.day[i]))
        self.dataframe['EventDate'] = eventdate
        #suppose there is only one event in a month
        uniquedays = self.dataframe['EventDate'].unique()
        inter_days = []
        for i in range(0,len(uniquedays)-1):
            inter_days.append(int((uniquedays[i+1]-uniquedays[i])/np.timedelta64(1,'D')))
        self.sample_length = min(inter_days)
    
    def reg(self):
        eventdate_set = self.dataframe['EventDate'].unique()
        subdate_cumulative_abnormal_return = pd.DataFrame()
        for date in eventdate_set:
            subevent_df = self.dataframe[self.dataframe.EventDate == date]
            start_date = date - np.timedelta64(self.sample_length, 'D')
            end_date = date + np.timedelta64(self.sample_length, 'D')
            subevent_company_list = subevent_df['Ticker'].unique()
            subevent_us_company_list = list(subevent_df[subevent_df['Country'] == 'U.S.']['Ticker'].unique())
            subevent_china_company_list = list(subevent_df[subevent_df['Country'] == 'China']['Ticker'].unique())
            subevent_cumulative_abnormal_return = pd.DataFrame()
            
            for company in subevent_company_list:
                market = subevent_df[subevent_df.Ticker == company]['Market'].iloc[0]
                price = web.DataReader([company, market],'yahoo', start_date,end_date)
                close_price = price['Close']
                return_rate = close_price.pct_change()
                x = return_rate[company]    
                X = sm.add_constant(x)
                y = return_rate[market]
                model = sm.OLS(y,X,missing='drop')
                result = model.fit()
#                 print(result.summary())
                (a,b) = result.params

                return_rate['Abnormal_Return'] = return_rate.apply(lambda x: x[market] - a - b * x[company], axis=1)
                subevent_cumulative_abnormal_return[f'{company}'] = return_rate['Abnormal_Return'].cumsum()[1:]
            subevent_cumulative_abnormal_return['Average_CAR'] = subevent_cumulative_abnormal_return.apply(lambda x: x.sum()/(len(subevent_company_list)),axis = 1)
                
                
            subevent_cumulative_abnormal_return_us_company = subevent_cumulative_abnormal_return[subevent_us_company_list]
            subevent_cumulative_abnormal_return_us_company['Average_CAR'] = subevent_cumulative_abnormal_return_us_company.apply(lambda x: x.sum()/(len(subevent_us_company_list)),axis = 1)
                
            subevent_cumulative_abnormal_return_china_company = subevent_cumulative_abnormal_return[subevent_china_company_list]
            subevent_cumulative_abnormal_return_china_company['Average_CAR'] = subevent_cumulative_abnormal_return_china_company.apply(lambda x: x.sum()/(len(subevent_china_company_list)),axis = 1)
            self.reg_result.append(subevent_cumulative_abnormal_return['Average_CAR'])
            self.reg_result_us.append(subevent_cumulative_abnormal_return_us_company['Average_CAR'])
            self.reg_result_china.append(subevent_cumulative_abnormal_return_china_company['Average_CAR'])
        
        self.us_return = sum([i[-1] for i in self.reg_result_us])
        self.china_return = sum([i[-1] for i in self.reg_result_china])
            
    def visualization(self):
        COL_NUM = 2
        ROW_NUM = len(instance.reg_result)//2+1
        fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(200,200))

        for i in range(len(self.reg_result)):
            ax = axes[int(i/COL_NUM), i%COL_NUM]
            self.reg_result[i].plot(kind='bar', ax=ax)
        
        for i in range(len(self.reg_result)):
            ax = axes[int(i/COL_NUM), i%COL_NUM]
            self.reg_result_china[i].plot(kind='line', ax=ax, c='black',title='China', linewidth=20)
            self.reg_result_us[i].plot(kind='line', ax=ax, c='blue', title='US', linewidth=20)
            ax.set_title(str('event'+str(i)))
       
        plt.bar(list(range(2)),[self.us_return, self.china_return], align='center')
        plt.xticks(list(range(2)), ['U.S.','China'])
        # plt.xticks(rotation=90)
        plt.title("Cumulative Abnormal Returns for U.S. & China Portfolios")
        plt.ylabel('return %')
        plt.show()

In [ ]:
#General Visualization
fed_path = '/Users/sheng/Documents/Jupyter Notebook/Tools for Analytics/Fall 2018/Fed events v3.csv'
trade_path = '/Users/sheng/Documents/Jupyter Notebook/Tools for Analytics/Fall 2018/Trade disputes events v3.csv'

fed_dataframe = pd.read_csv(fed_path)
trade_dataframe = pd.read_csv(trade_path)

fed_event_name = fed_dataframe['Event']
fed_company_name = fed_dataframe['Company Name']
fed_company = fed_dataframe['Ticker'] # ticker name
fed_market = fed_dataframe['Market']
us_industry = fed_dataframe['Industry']
fed_year = fed_dataframe['Year']
fed_month = fed_dataframe['Month']
fed_day = fed_dataframe['Day']

trade_event_name = trade_dataframe['Event']
trade_company_name = trade_dataframe['Company Name']
trade_company = trade_dataframe['Ticker'] # ticker name
trade_market = trade_dataframe['Market']
trade_industry = trade_dataframe['Industry']
trade_year = trade_dataframe['Year']
trade_month = trade_dataframe['Month']
trade_day = trade_dataframe['Day']
trade_country = trade_dataframe['Country']

# Fed data processing

fed_dic = {}
for i in range(0, len(fed_market)):
    fed_dic[fed_company[i]] = fed_market[i]

# Add an "EventDate" column to the main dataframe
fed_eventdate = []
for i in range(0, len(fed_dataframe)):
    fed_eventdate.append(datetime.datetime(fed_year[i], fed_month[i], fed_day[i]))
fed_dataframe['EventDate'] = fed_eventdate

# fed_dataframe
fed_inter_days = []
fed_uniquedays = sorted(fed_dataframe['EventDate'].unique()) # 8 unique days
for i in range(0, len(fed_uniquedays)-1):
    fed_inter_days.append(int((fed_uniquedays[i+1] - fed_uniquedays[i])/np.timedelta64(1,'D')))
fed_sample_length = min(fed_inter_days) # 84, inter_days:  [364, 91, 91, 182, 98, 84, 105]
# fed_sample_length
fed_eventdate_set = fed_dataframe['EventDate'].unique()

# Trade data processing

trade_dic = {}
for i in range(0, len(trade_market)):
    trade_dic[trade_company[i]] = trade_market[i]

# Add an "EventDate" column to the main dataframe
trade_eventdate = []
for i in range(0, len(trade_dataframe)):
    trade_eventdate.append(datetime.datetime(trade_year[i], trade_month[i], trade_day[i]))
trade_dataframe['EventDate'] = trade_eventdate

# trade_dataframe
trade_inter_days = []
trade_uniquedays = sorted(trade_dataframe['EventDate'].unique()) # 8 unique days
for i in range(0, len(trade_uniquedays)-1):
    trade_inter_days.append(int((trade_uniquedays[i+1] - trade_uniquedays[i])/np.timedelta64(1,'D')))
trade_sample_length = min(trade_inter_days) # 84, inter_days:  [364, 91, 91, 182, 98, 84, 105]
# fed_sample_length
trade_eventdate_set = trade_dataframe['EventDate'].unique()
# len(trade_eventdate_set)

# Creating inputs for data overview

num_fed_events = len(fed_eventdate_set) # 8
num_trade_events = len(trade_eventdate_set) # 24
event_cat_label = ['Fed Events', 'Trade Dispute Events']# num_trade_events

num_us_stock = len(us_industry)/num_fed_events #30
num_china_stock = len(trade_industry)/num_trade_events - num_us_stock #10
fed_event_list = list(fed_event_name[[0 + x*(len(fed_dataframe)-0)/len(fed_eventdate_set) for x in range(len(fed_eventdate_set))]])
trade_event_list = list(trade_event_name[[0 + x*(len(trade_dataframe)-0)/len(trade_eventdate_set) for x in range(len(trade_eventdate_set))]])
# trade_event_list

us_industry_list = list(us_industry)[0:int(num_us_stock)] #30 with duplicates
us_unique_industry_list = list(set(us_industry_list)) #12 distinct industries
us_unique_industry_count = list(np.unique(us_industry_list, return_counts=True)[1]) # 12 numbers
us_unique_industry_count_pct = [(x*100) / 30 for x in us_unique_industry_count]

china_industry_list = list(trade_industry)[int(num_us_stock):int(num_us_stock+num_china_stock)] #10
china_unique_industry_list = list(set(china_industry_list)) # 8
china_unique_industry_count = list(np.unique(china_industry_list, return_counts=True)[1]) # 8 numbers
china_unique_industry_count_pct = [(x*100) / 30 for x in china_unique_industry_count]

combined_industry_list = list(trade_industry)[0:int(num_us_stock+num_china_stock)] #40 with duplicates
combined_unique_industry_list = list(set(combined_industry_list)) #13 distinct industries
combined_unique_industry_count = list(np.unique(combined_industry_list, return_counts=True)[1])
combined_unique_industry_count_pct = [(x*100) / 30 for x in combined_unique_industry_count]

# us_stock_list = list(company)[0:int(num_us_stock)]
# us_company_list = list(dataframe['Company Name'])[0:int(num_us_stock)]

# 5 plots for event, stock, and industry compositions
import matplotlib.gridspec as gridspec

fig = plt.figure(1)
gridspec.GridSpec(4,2) # 4 rows, 2 columns

# small plot at (0,0)
plt.subplot2grid((4,2), (0,0))
plt.pie([num_fed_events, num_trade_events], labels = event_cat_label, autopct = '%1.1f%%')
plt.title('Event Composition')

# small plot at (0,1)
plt.subplot2grid((4,2), (0,1))
plt.pie([num_us_stock, num_china_stock], labels = ['U.S. Stocks', 'Chinese Stocks'], autopct = '%1.1f%%')
plt.title('Stock Composition')

# small plot at (1,0)
plt.subplot2grid((4,2), (1,0))
plt.pie(us_unique_industry_count_pct,labels = us_unique_industry_list, autopct = '%1.1f%%')
plt.title('U.S. Stock Industry Composition')

# small plot at (1,1)
plt.subplot2grid((4,2), (1,1))
plt.pie(china_unique_industry_count_pct, labels= china_unique_industry_list, autopct = '%1.1f%%')
plt.title('Chinese Stock Industry Composition')

# large plot at (2,0)
plt.subplot2grid((4,2), (2,0), colspan=2, rowspan=2)
plt.pie(combined_unique_industry_count_pct, labels= combined_unique_industry_list, autopct = '%1.1f%%')
plt.title('Combined Industry Composition')

fig.set_size_inches(w=18,h=35)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
# labels = 'Frogs', 'Hogs', 'Dogs', 'Logs'
# sizes = [15, 30, 45, 10]
# explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
ax1.pie(us_unique_industry_count_pct, labels=us_unique_industry_list, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()